In [1]:
import sys
sys.path.append("modules/")
import scraper
from bs4 import BeautifulSoup

html = scraper.fetch_html("https://www.news18.com/", waitUntil='static')
soup = BeautifulSoup(html, 'html.parser')
soup = scraper.clean_html(soup)
soup = scraper.squash_nested_divs(soup)
tags = scraper.extract_tags(soup)

for i, tag in enumerate(tags):
    print(f'{i}: {tag}\n')

0: <a class="jsx-7ca0ea2315bd3cb3 logo" href="https://www.news18.com/"><img alt="News18 - Latest News" class="jsx-7ca0ea2315bd3cb3" src="https://images.news18.com/static_news18/pix/ibnhome/news18/News18.svg" style="width:130px;min-height:47px;max-height:80px" title="News18 - Latest News"/></a>

1: <a class="jsx-fb06aeb20aaa3410" href="https://www.news18.com/">English<!-- --> <span class="jsx-fb06aeb20aaa3410 nhlanguate-arrow hsocial-sprite"></span></a>

2: <a class="jsx-fb06aeb20aaa3410" href="https://hindi.news18.com/" rel="noreferrer" target="_blank">हिन्दी<!-- --> </a>

3: <a class="jsx-fb06aeb20aaa3410" href="https://bengali.news18.com/" rel="noreferrer" target="_blank">বাংলা<!-- --> </a>

4: <a class="jsx-fb06aeb20aaa3410" href="https://news18marathi.com/" rel="noreferrer" target="_blank">मराठी<!-- --> </a>

5: <a class="jsx-fb06aeb20aaa3410" href="https://gujarati.news18.com/" rel="noreferrer" target="_blank">ગુજરાતી<!-- --> </a>

6: <a class="jsx-fb06aeb20aaa3410" href="https://

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

checkpoint_path = "models/classifier_model/"  # Path to your fine-tuned model
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path, use_auth_token=True)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)

/home/mananj2/miniconda3/envs/wc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mananj2/miniconda3/envs/wc/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
2024-01-07 15:23:15.477815: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-07 15:23:15.477883: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-07 15:23:15.479484: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:

In [8]:
import torch
from torch.nn.functional import softmax

def preprocess_inference(text):
    inputs = tokenizer(text, truncation=True, return_tensors="pt")
    return inputs

def predict(text):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient calculation for inference
        inputs = preprocess_inference(text)
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = softmax(logits, dim=1)
        return probabilities
    
def get_prediction(text):
    probabilities = predict(text)
    predicted_class = torch.argmax(probabilities, dim=1).item()
    return predicted_class  # You can map this to a class label if you have a label mapping

In [9]:
articles = []
not_articles = []
for tag in tags:
    prediction = get_prediction(tag)
    if prediction:
        articles.append(tag)
    else:
        not_articles.append(tag)

In [10]:
for a in articles:
    print(a, '\n')

<a class="jsx-7ca0ea2315bd3cb3 logo" href="https://www.news18.com/"><img alt="News18 - Latest News" class="jsx-7ca0ea2315bd3cb3" src="https://images.news18.com/static_news18/pix/ibnhome/news18/News18.svg" style="width:130px;min-height:47px;max-height:80px" title="News18 - Latest News"/></a> 

<a class="jsx-eec62120448a2e6d" data-href="https://www.news18.com/india/51-inches-tall-weighing-1-5-tonnes-and-has-the-innocence-of-a-child-all-about-ram-lalla-idol-at-ram-mandir-8730258.html" href="" id="para1">51 Inches Tall, Weighing 1.5 Tonnes, and 'Has the Innocence of a Child' | All About Ram Lalla Idol at Ram Mandir</a> 

<a href="https://www.news18.com/world/maldives-ministers-social-media-post-stirs-controversy-over-pm-modis-lakshadweep-visit-8730914.html" target="_blank"><img alt="" height="360px" priority="1" src="https://images.news18.com/ibnlive/uploads/2024/01/untitled-design-1-2024-01-4680fea80c051c6600a21a5d07994391-3x2.png?impolicy=website&amp;width=540&amp;height=360" title="" wi

In [12]:
print(len(articles))
print(len(not_articles))

205
343
